### Fine Tuning Homework with Unsloth

Please refer to: https://docs.unsloth.ai/get-started/fine-tuning-llms-guide

# 0: Setup

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.57.0
!pip install --no-deps trl==0.22.2
!pip install evaluate

Plz mount your google drive.

And upaload this notebook on the drive's foloder 'fine-tune-tutorial'.

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
work_path = '/gdrive/My Drive/fine-tune-tutorial'

NOTE: you can modify the folder as you want

### Specify the pretrained model as `LLama-3.1-8B-Instruct-bnb-4bit``

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

LoRA Setup: Parameter-Efficient Fine-Tuning (PEFT)

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.



# 1: Data preparation


Use the given dataset `order_analysis-dataset.json`

In [6]:
from datasets import load_dataset

# Load the dataset from the JSON file
dataset = load_dataset("json", data_files=f"{work_path}/order_analysis-dataset.json", split='train')


Generating train split: 0 examples [00:00, ? examples/s]

##  Q1: Split the dataset into training and test sets using an 80:20 ratio.

In [7]:
dataset = dataset.train_test_split(
    test_size=0.2,
    seed=42
)

### 원본 데이터셋 확인

In [8]:
print(dataset.column_names)

{'train': ['instruction', 'input', 'output'], 'test': ['instruction', 'input', 'output']}


In [9]:
print(dataset['train'][0])

{'instruction': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.', 'input': '주문 문장: 아이스 바닐라라떼 한잔하고 땡참김밥 둘 주시겠어요?', 'output': '- 분석 결과 0: 음식명:아이스 바닐라라떼,수량:한잔\n- 분석 결과 1: 음식명:땡참김밥,수량:둘'}


### Q2: Transform dataset's format int ChatML

instruction -> system

input -> user

output -> assistant

```json
[
    {"role": "system", "content": system},
    {"role": "user", "content": input},
    {"role": "assistant", "content": output}
]
```

In [10]:
def convert_to_chatml_format(examples):
    systems = examples['instruction']
    inputs = examples['input']
    outputs = examples['output']

    texts = [
        [
            {"role": "system", "content": s},
            {"role": "user", "content": i},
            {"role": "assistant", "content": o},
        ]
        for s, i, o in zip(systems, inputs, outputs)
    ]

    return {"conversations": texts}

In [11]:
dataset = dataset.map(convert_to_chatml_format, batched=True)

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

You can see that the messages are organized under the 'conversations' key for every sample in the dataset.

```json
'conversations': [
    {'content': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
   'role': 'system'},
    {'content': '주문 문장: 아인슈페너 레귤러 사이즈로 부탁드리고, 콩국수 한 그릇 주세요.', 'role': 'user'},
    {'content': '- 분석 결과 0: 음식명:아인슈페너,옵션:레귤러,수량:1\n- 분석 결과 1: 음식명:콩국수,수량:한 그릇',
   'role': 'assistant'}]
```

In [12]:
dataset['train'][0]

{'instruction': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
 'input': '주문 문장: 아이스 바닐라라떼 한잔하고 땡참김밥 둘 주시겠어요?',
 'output': '- 분석 결과 0: 음식명:아이스 바닐라라떼,수량:한잔\n- 분석 결과 1: 음식명:땡참김밥,수량:둘',
 'conversations': [{'content': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
   'role': 'system'},
  {'content': '주문 문장: 아이스 바닐라라떼 한잔하고 땡참김밥 둘 주시겠어요?', 'role': 'user'},
  {'content': '- 분석 결과 0: 음식명:아이스 바닐라라떼,수량:한잔\n- 분석 결과 1: 음식명:땡참김밥,수량:둘',
   'role': 'assistant'}]}

### Q3: Apply `apply_chat_template`

Roles:
- Special tokens representing system, user, and assistant were added.
- These special tokens are implemented differently across various models, so a unified interface, apply_chat_template, is used.

In [13]:
from unsloth import apply_chat_template

dataset = apply_chat_template(dataset, tokenizer)

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

You can see the special tokes added

In [14]:
dataset['train'][0]

{'instruction': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
 'input': '주문 문장: 아이스 바닐라라떼 한잔하고 땡참김밥 둘 주시겠어요?',
 'output': '- 분석 결과 0: 음식명:아이스 바닐라라떼,수량:한잔\n- 분석 결과 1: 음식명:땡참김밥,수량:둘',
 'conversations': [{'content': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
   'role': 'system'},
  {'content': '주문 문장: 아이스 바닐라라떼 한잔하고 땡참김밥 둘 주시겠어요?', 'role': 'user'},
  {'content': '- 분석 결과 0: 음식명:아이스 바닐라라떼,수량:한잔\n- 분석 결과 1: 음식명:땡참김밥,수량:둘',
   'role': 'assistant'}],
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n주문 문장: 아이스 바닐라라떼 한잔하고 땡참김밥 둘 주시겠어요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n- 분석 결과 0: 음식명:아이스 바닐라라떼,수량:한잔\n- 분석 결과 1: 음식명:땡참김밥,수량:둘<|eot_id|>'}

In [15]:
tokenizer.apply_chat_template(dataset['train']['conversations'][0], tokenize = False, add_generation_prompt = False)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n주문 문장: 아이스 바닐라라떼 한잔하고 땡참김밥 둘 주시겠어요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n- 분석 결과 0: 음식명:아이스 바닐라라떼,수량:한잔\n- 분석 결과 1: 음식명:땡참김밥,수량:둘<|eot_id|>'

#2: Training the Model
The model is trained using Hugging Face TRL's `SFTTrainer`.

For more detailed information, please refer to the TRL SFT docs.

Due to time constraints, we will only run 60 steps.

When performing actual fine-tuning:
- Set num_train_epochs=1 or more.
- Set max_steps=None.

In [16]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'].select(range(1000)),
    dataset_text_field = "text",
    max_seq_length = 512,
    packing = True, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 2,
        #max_steps = 60, None으로 처리하여 epoch 차이를 확인
        num_train_epochs = 1,
        learning_rate = 1e-4,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.883 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Step,Training Loss
10,1.790800
20,0.799800
30,0.767400
40,0.622100
50,0.674900
60,0.564800
70,0.556800
80,0.625900
90,0.587200
100,0.712300


In [19]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1120.2187 seconds used for training.
18.67 minutes used for training.
Peak reserved memory = 7.303 GB.
Peak reserved memory for training = 0.42 GB.
Peak reserved memory % of max memory = 49.542 %.
Peak reserved memory for training % of max memory = 2.849 %.


# 3: Applying trained model

Apply the trained model to check if it has been successfully fine-tuned.


In [20]:
system_message = '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.'

In [21]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "짜장면 2그릇, 콜라 1병 주세요."},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


- 분석 결과 0: 음식명:짜장면,수량:2그릇
- 분석 결과 1: 음식명:콜라,수량:1병<|eot_id|>


# 4: Saving and Loading the Fine-Tuned Model
We only save the LoRA adapter (for efficient storage):
- Save to the Hugging Face Hub: `push_to_hub`
- Save locally: `save_pretrained`

In [22]:
model.save_pretrained(f"{work_path}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{work_path}/lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/gdrive/My Drive/fine-tune-tutorial/lora_model/tokenizer_config.json',
 '/gdrive/My Drive/fine-tune-tutorial/lora_model/special_tokens_map.json',
 '/gdrive/My Drive/fine-tune-tutorial/lora_model/chat_template.jinja',
 '/gdrive/My Drive/fine-tune-tutorial/lora_model/tokenizer.json')

# 5: Loading the saved model and tokenizer

In [23]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = f"{work_path}/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [24]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 256, pad_token_id = tokenizer.eos_token_id)

- 분석 결과 0: 음식명:짜장면, 수량:2그릇
- 분석 결과 1: 음식명:콜라, 수량:1병<|eot_id|>


We can also load the saved model as Hugging Face style
- `AutoModelForPeftCausalLM`.
- `AutoTokenizer`

In [25]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        f"{work_path}/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained(f"{work_path}/lora_model")

#

# 6: Calculate Metrics

### Q4: Complte the code for calculating BLEU score on the test split

In [26]:
import evaluate
bleu_metric = evaluate.load("bleu")

In [27]:
test_dataset = dataset['test']

generated_responses = []
reference_responses = []

for i, example in enumerate(test_dataset):
    if i >= 100:
        break

    prompt = tokenizer.apply_chat_template(
        example["conversations"][:-1],
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.0,
        do_sample=False,
    )

    input_length = inputs["input_ids"].shape[1]
    generated_text = tokenizer.decode(
        outputs[0][input_length:],
        skip_special_tokens=True,
    ).strip()

    reference_text = example["output"].strip()

    generated_responses.append(generated_text.strip())
    reference_responses.append(reference_text.strip()) # Use the original output as reference

In [30]:
result_1 = bleu_metric.compute(predictions=generated_responses, references=reference_responses)

In [31]:
print(result_1['bleu'])

0.9173199272743199


# 7: BONUS Question (*)

### Q5: Try additional efforts to get the best bleu score on the test split (Plz submit another notebook for this Bonus question)
- Option 1: Train more epochs
- Option 2: Try other models rather than Meta-Llama-3.1-8B-Instruct-bnb-4bit

# - epoch 조정

**- 환경 변수 변경**

Colab 환경의 자원 제약을 고려하며, BLEU 성능 향상을 목표로 추가 미세조정을 수행하였다. GPU 메모리를 절약하기 위해 batch size는 2에서 1로 줄였으며, gradient accumulation은 4에서 1로 축소하였다. 또한 warmup step은 5에서 2로 낮추어 전체 연산량을 줄였고, 학습 시간이 과도하게 길어지는 것을 방지하기 위해 max_steps는 제거하였다. 과적합을 최소화하기 위해 learning rate는 2e-4에서 1e-4로 조정하였다.

이러한 설정을 고정한 상태에서 num_train_epochs = 1, 2, 3으로 순차적으로 실험을 진행하여 epoch 증가가 BLEU 성능에 미치는 영향을 분석하였다. 모든 실험은 max_seq_length = 512, packing = True로 동일하게 진행했다.


**- BLEU Score Results**
1. epoch = 1인 상황에서 bleu 점수는 :0.9150880591680964
2. epoch = 2인 상황에서 bleu 점수는 :0.9210181790206565
3. epoch = 3인 상황에서 bleu 점수는 :0.9064429358819918

성능 : epoch 2 > epoch1 > epoch 3

Epoch을 1에서 2로 증가시켰을 때 BLEU가 상승한 것은 모델이 파인튜닝을 적절히 수행한 것으로 보인다. 그러나 epoch을 3까지 늘리자 오히려 점수가 하락했으며, 이는 과도한 반복 학습으로 인해 과적합되었기 때문이다. Epoch 3에서는 다양성이 감소하고 특정 표현 방식에 지나치게 맞춰지면서 전체 성능이 떨어지는 결과를 보였다.

# - LoRA 조정


다음으로는, 성능이 최대인 epoch=2 상황에서 LoRA 조정을 통한 성능 향상을 목표로 미세조정을 수행하였다.
r = 16 ,lora_alpha = 16

**- BLEU Score Results**
1. r = 8, lora_alpha = 8인 상황에서 bleu score = 0.9138349478136333
2. r = 16 ,lora_alpha = 16인 상황에서 bleu score = 0.9210181790206565
3. r = 32, lora_alpha = 32인 상황에서 bleu score =0.9222873616581291
4. r = 64, lora_alpha = 64인 상황에서 bleu score =0.9173199272743199

성능 : r= 32 > r= 16 > r= 64 > r= 8

LoRA Rank를 8, 16, 32, 64로 변화시키며 성능을 비교한 결과, r이 증가할수록 모델의 표현력이 확장되어 r=32까지는 성능이 지속적으로 향상되었다. 그러나 r=64에서는 점수가 하락했는데, 이는 학습 가능한 파라미터가 늘어나 데이터의 패턴 등이 적합되었기 때문이다. 즉, LoRA는 적정 범위(약 16~32)에서는 성능을 개선하지만, 이를 초과하면 일반화 성능이 떨어져 오히려 성능이 감소하는 경향을 보이며, 본 실험에서는 r=32가 가장 효율적인 설정으로 나타났다.